In [1]:
import numpy as np 
import re, nltk
from numpy import array 
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense 
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
df1= pd.read_csv(r'ALL_language.csv', encoding = "utf-8")
df = df1.sample(frac =1)
display(df)

,English,Amharic,Lithuanian,Arabic,Germany,Tigrinya,French,Czech,Intent
16599,set alarm every 10 minutes for 20 minutes,ለ 20 ደቂቃዎች በየደረጃው ሁሉንም 10 ደቂቃዎችን ያዘጋጁ,Nustatykite aliarmą kas 10 minučių 20 minučių,اضبط المنبه كل 10 دقائق لمدة 20 دقيقة,Stellen Sie alle 10 Minuten für 20 Minuten Ala...,NaN,régler l'alarme toutes les 10 minutes pendant ...,Nastavte alarm každých 10 minut po dobu 20 minut,alarm__set_alarm
11613,did it rain yesterday ?,ትናንት ዝናብን ዝናብ አደረገ?,Ar vakar lietus?,هل تمطر أمس؟,hat es gestern geregnet ?,NaN,Est-ce qu'il a plu hier ?,pršelo včera?,weather__find
19093,is it going to get cold tonight ?,ዛሬ ማታ ቀዝቃዛ ነው?,Ar šį vakarą bus šalta?,هل ستصبح باردة الليلة؟,Wird es heute Abend kalt?,NaN,Est-ce que ça va faire froid ce soir?,Bude to dnes večer zchladnout?,weather__find
19666,remove all alarms on sundays,እሑድ ላይ ሁሉንም ማንቂያዎች ያስወግዱ,Išimkite visus aliarmus sekmadieniais,إزالة جميع الإنذارات يوم الأحد,Entfernen Sie sonntags alle Alarme,NaN,Retirez toutes les alarmes le dimanche,Odstraňte všechny alarmy v neděli,alarm__cancel_alarm
10697,delete every alarm that i have set .,ያዘጋጃቸውን ደወል እያንዳንዱን ደወል ሰርዝ.,Ištrinkite kiekvieną mano nustatytą aliarmą.,حذف كل إنذار قمت بتعيينه.,"Löschen Sie jeden Alarm, den ich gesetzt habe.",NaN,Supprimez chaque alarme que j'ai définie.,"Odstraňte každý alarm, který jsem nastavil.",alarm__cancel_alarm
...,...,...,...,...,...,...,...,...,...
16837,when should i start packing away my summer clo...,የበጋዎትን ልብሶቼን ማሸግ ያለብኝ መቼ ነው?,Kada turėčiau pradėti supakuoti savo vasaros d...,متى يجب أن أبدأ في تعبئة ملابسي الصيفية؟,"Wann sollte ich anfangen, meine Sommerkleidung...",NaN,Quand dois-je commencer à emballer mes vêtemen...,Kdy bych měl začít balit své letní oblečení?,weather__find
7343,what is the temperature outside today ?,ከዛሬ ውጭ ያለው የሙቀት መጠን ምንድነው?,Kokia temperatūra šiandien lauke?,ما هي درجة الحرارة في الخارج اليوم؟,Was ist die Temperatur heute draußen?,NaN,Quelle est la température à l'extérieur aujour...,Jaká je dnes teplota venku?,weather__find
15031,set my alarm for 4 am tomorrow .,ነገን ለ 4 ሰዓት ያዘጋጁ.,Nustatykite savo žadintuvą rytoj 4 val.,اضبط إنذاري لمدة 4 صباحًا غدًا.,Stellen Sie meinen Alarm morgen für 4 Uhr ein.,NaN,Définissez mon alarme pour 4 heures du matin d...,Zítra nastavte můj alarm na 4:00.,alarm__set_alarm
998,how long until alarm on timer goes off,ሰዓት ቆጣሪው እስኪወጣ ድረስ ምን ያህል ጊዜ,"Kiek laiko, kol bus išeis laikmatis",كم من الوقت حتى ينفجر المنبه في المؤقت,"Wie lange dauert es, bis der Alarm am Timer au...",NaN,Combien de temps jusqu'à ce que l'alarme de la...,"Jak dlouho dojde k tomu, aby se zhroutil alarm...",alarm__time_left_on_alarm


In [3]:
sentence = df.French.values.tolist()
sentiment = df.Intent.values.tolist()
print(len(sentence))
print(len(sentiment))

22994
22994


In [4]:
import pandas as pd
df2= pd.read_csv(r'Prompt2chatgpt.csv', encoding = "utf-8")
df3 = df2.sample(frac =1)
display(df3)

,English,Amharic,Germany,Lithuania,French,Czech,Intent
1,Move my wake-up alarms to 7:45 AM for the enti...,የኔ ማንቂያ ደውሎቶችን ለ 8:45 የሚቀጥለውን ሳምንት ማስወጣት ይችላሉ?,Wiederhole den Alarm bis zum Ende von 2017,Pakartokite pabudimą iki 2017 m. pabaigos,Répète l'alarme jusqu'à la fin de 2017,Opakujte alarm do konce roku 2017,alarm__modify_alarm
84,Snooze for 5 minutes.,SNOOZE ን ለ 12 ደቂቃዎች ያዘጋጁ,Stellen Sie den Snooze auf 10 Minuten ein,Nustatykite snaudimą 14 minučių,Réglez la répétition pendant 10 minutes,Nastavte odložení na 10 minut,alarm__snooze_alarm
153,When will the sun rise tomorrow on the east co...,ለምን የታመነ ዓመታዊ ሰዓት የፀሐይ መውጫ ምን ሰዓት ነው,Wie spät ist der Sonnenaufgang morgen?,Kada rytoj saulėtekis vyks?,Quelle heure est le lever du soleil demain,V kolik je čas na východ slunce,weather__checkSunrise
98,instance,እንደአንደኛው ፀሐይ ስትጠልቅ መቼ ነው,Wann geht die Sonne heute unter?,Kada yra šiandieninis saulėlydis?,Quand est le coucher du soleil,Kdy je západ slunce,weather__checkSunset
41,Change the alarm sound to a calming melody fro...,መልስ እስከዚህ ድረስ በየ 8 ደቂቃው ደወል,Wiederhole den Alarm bis zum Ende des Jahres 2017,"Pakartokite žadintuvą kas 12 minutes, kol bus ...",Répète l'alarme jusqu'à la fin de 2017,Opakujte alarm do konce roku 2017,alarm__modify_alarm
...,...,...,...,...,...,...,...
87,Please snooze for an additional 3 minutes.,SNOOZE ን ለ 30 ደቂቃዎች ያዘጋጁ,Setzen Sie den Alarm auf 15 Minuten Snooze,"Nustatykite aliarmą, kad snaudžiate dar 22 min...",Réglez l'alarme pour répéter pendant 15 minute...,"Nastavte alarm, aby se odložil po dobu dalších...",alarm__snooze_alarm
183,What time is the sunrise tomorrow?,ለምን የታመነ ዓመታዊ ሰዓት ምን ያህል ጊዜ ከፍ ብሏል?,Wie spät ist der Sonnenaufgang morgen?,Kada rytoj saulėtekis vyks?,Quelle heure est le lever du soleil demain,V kolik je čas na východ slunce,weather__checkSunrise
139,What time is the sunset in the summer tonight?,እንደ አንደኛው ዛሬ ማታ ፀሐይ መቼ ነው የሚያደርገው?,Wie spät geht die Sonne heute unter?,Kada šį vakarą saulė leisiasi?,À quelle heure le soleil se couche-t-il aujour...,V kolik dnes slunce zapadlo,weather__checkSunset
96,Activate snooze for an extra 6 minutes.,ለማሸለፊያ ሁኔታ ለ 18 ደቂቃዎች ያዘጋጁ,Snooze für zusätzliche 5 Minuten,Nuplėškite 30 minutes,Snooze pendant 5 minutes,Snooze po dobu 5 minut,alarm__snooze_alarm


In [5]:
sentence_1 = df3.Amharic.values.tolist()
sentiment_1 = df3.Intent.values.tolist()
print(len(sentence_1))
print(len(sentiment_1))

189
189


In [6]:
for a in sentence_1:
    sentence.append(a)
for b in sentiment_1:
    sentiment.append(b)

print(len(sentiment))
print(len(sentence))

23183
23183


In [7]:
print(len(df.Intent.unique()))
print(df.Intent.value_counts())

13
weather__find                10461
reminder__set_reminder        4081
alarm__set_alarm              3915
alarm__cancel_alarm           1307
reminder__cancel_reminder      813
alarm__show_alarms             705
reminder__show_reminders       600
alarm__modify_alarm            399
alarm__time_left_on_alarm      270
alarm__snooze_alarm            260
weather__checkSunset           103
weather__checkSunrise           79
alarm__set_alar                  1
Name: Intent, dtype: int64


In [8]:
sentrp = sentence[0:18560]
sentsp = sentence[18400:23196]
semtr = sentiment[0:18560]
semts = sentiment[18400:23196]
print(len(sentrp))
print(len(sentsp))
print(len(semtr))
print(len(semts))


18560
4783
18560
4783


In [9]:
print(sentrp[0:10])
print(sentsp[0:10])
print(semtr[0:10])
print(semts[0:10])

["régler l'alarme toutes les 10 minutes pendant 20 minutes", "Est-ce qu'il a plu hier ?", 'Est-ce que ça va faire froid ce soir?', 'Retirez toutes les alarmes le dimanche', "Supprimez chaque alarme que j'ai définie.", 'Donnez-moi une alarme en 15 minutes.', 'Veuillez effacer tous les rappels.', 'À quelle vitesse sont les vents de cet ouragan', 'Supprimez toutes les alarmes faites dans ce mois-ci.', 'Y aura-t-il du vent ce soir']
['Quelle est la basse température pour ce soir', "Réglez l'alarme pour 8 demain.", "Effacer toutes les alarmes enregistrées s'il vous plaît", "Est-ce qu'il pleut à St Louis en ce moment", "Est-ce que ça va être chaud aujourd'hui?", 'Alarme sonore toutes les 20 minutes', 'Y aura-t-il un blizzard demain', "Est-il censé être très chaud et humide aujourd'hui?", 'Montre-moi mes alarmes actuelles', 'sera-t-il supérieur à 41 en degrés Celsius pour Karachi']
['alarm__set_alarm', 'weather__find', 'weather__find', 'alarm__cancel_alarm', 'alarm__cancel_alarm', 'alarm__set

In [10]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
model = SentenceTransformer('sentence-transformers/LaBSE')
embeddingtr = model.encode(sentrp, batch_size = 128, convert_to_numpy = True, show_progress_bar = True)
embeddingts = model.encode(sentsp, batch_size = 128, convert_to_numpy = True, show_progress_bar = True)

Batches:   0%|          | 0/145 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

print(len(embeddingts))
print(len(embeddingtr))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
frames = []
for idx, val in enumerate(sentsp):
    cos_va = cosine_similarity(embeddingts, embeddingtr)[idx]
    df1 = pd.DataFrame(sentrp, columns =['sentenc'])
    df1['position'] = df1.index.values
    
    df1['realsem'] = semts[idx]
    df1['new_cosine'] = (cos_va)
    query_sentence_cosine_df1 = df1.sort_values('new_cosine', ascending = False)
    df2 = query_sentence_cosine_df1[['sentenc', 'new_cosine','position','realsem']].head(3)
    frames.append(df2)
    result = pd.concat(frames)
display(result)

In [ ]:
for i in result['position']:
    result['pred'][i]= semtr[i]
display(result)

In [37]:
import numpy
Grp = []
subs = []
result['pred']
for i in result['pred']:
    Grp.append(i)
#print(Grp)
print(len(Grp))
print(type(Grp))
splits = np.array_split(Grp,len(sentsp))# no of testing sets
vote = []
for array in splits:
    vote.append(list(array))
print(vote)
gr = []
for el in vote:
    mx = max(set(el), key =el.count)
    gr.append(mx)
    
print(gr)
print(type(gr))
print(len(gr))

14349
<class 'list'>
[['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['alarm__set_alarm', 'alarm__modify_alarm', 'alarm__cancel_alarm'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['alarm__snooze_alarm', 'alarm__cancel_alarm', 'alarm__snooze_alarm'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['weather__find', 'weather__find', 'weather__find'], ['alarm__set_alarm', 'alarm__set_alarm', 'alarm__set_alarm'], ['weather__find', 'weather__find', 'weather__find'], ['reminder__set_reminder', 'reminder__show_reminders', 'reminder__show_reminders'], ['reminder__set_reminder', 'reminder__set_reminder', 'rem

In [38]:
from sklearn.metrics.pairwise import cosine_similarity
Fram = []
for idx, val in enumerate(sentsp):
    cos_va = cosine_similarity(embeddingts, embeddingtr)[idx]
    
    df1 = pd.DataFrame(sentrp, columns =['sentenc'])
    df1['position'] = df1.index.values
    df1['realsem'] = semts[idx]
    df1['new_cosine'] = (cos_va)
    query_sentence_cosine_df1 = df1.sort_values('new_cosine', ascending = False)
    res = query_sentence_cosine_df1[['sentenc', 'new_cosine','position','realsem']].head(1)
    Fram.append(res)
    reslt1= pd.concat(Fram)
display(reslt1)

,sentenc,new_cosine,position,realsem
18400,Quel temps fait-il pour Chicago demain?,1.000000,18400,weather__find
18401,Une pluie attendue la semaine prochaine?,1.000000,18401,weather__find
18402,Quel temps à la plage,1.000000,18402,weather__find
18403,faire couler mon alarme dans une heure,1.000000,18403,alarm__set_alarm
18404,Comme c'est froid pour aujourd'hui,1.000000,18404,weather__find
...,...,...,...,...
5541,Réglez l'alarme à 15 minutes,0.937903,5541,alarm__snooze_alarm
8981,Ajouter 10 minutes à mon alarme,0.978060,8981,alarm__modify_alarm
5541,Réglez l'alarme à 15 minutes,0.937903,5541,alarm__snooze_alarm
7491,À quelle heure le soleil se lève-t-il ce matin,0.911185,7491,weather__checkSunrise


In [39]:
reslt1['predic']= gr
display(reslt1) 

,sentenc,new_cosine,position,realsem,predic
18400,Quel temps fait-il pour Chicago demain?,1.000000,18400,weather__find,weather__find
18401,Une pluie attendue la semaine prochaine?,1.000000,18401,weather__find,weather__find
18402,Quel temps à la plage,1.000000,18402,weather__find,weather__find
18403,faire couler mon alarme dans une heure,1.000000,18403,alarm__set_alarm,alarm__set_alarm
18404,Comme c'est froid pour aujourd'hui,1.000000,18404,weather__find,weather__find
...,...,...,...,...,...
5541,Réglez l'alarme à 15 minutes,0.937903,5541,alarm__snooze_alarm,alarm__set_alarm
8981,Ajouter 10 minutes à mon alarme,0.978060,8981,alarm__modify_alarm,alarm__modify_alarm
5541,Réglez l'alarme à 15 minutes,0.937903,5541,alarm__snooze_alarm,alarm__set_alarm
7491,À quelle heure le soleil se lève-t-il ce matin,0.911185,7491,weather__checkSunrise,weather__checkSunrise


In [40]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(reslt1)

,sentenc,new_cosine,position,realsem,predic
18400,Quel temps fait-il pour Chicago demain?,1.000000,18400,weather__find,weather__find
18401,Une pluie attendue la semaine prochaine?,1.000000,18401,weather__find,weather__find
18402,Quel temps à la plage,1.000000,18402,weather__find,weather__find
18403,faire couler mon alarme dans une heure,1.000000,18403,alarm__set_alarm,alarm__set_alarm
18404,Comme c'est froid pour aujourd'hui,1.000000,18404,weather__find,weather__find
18405,Quelle est la température de Buffalo?,1.000000,18405,weather__find,weather__find
18406,Comme ça va faire chaud,1.000000,18406,weather__find,weather__find
18407,Ava-t-il atteindre 70 degrés demain?,1.000000,18407,weather__find,weather__find
18408,Dites-moi le temps à Tucson en ce moment.,1.000000,18408,weather__find,weather__find
18409,Pouvez-vous somnoler l'alarme,1.000000,18409,alarm__snooze_alarm,alarm__snooze_alarm


In [41]:
import pandas as pd
true_class = reslt1["realsem"].to_numpy()
pred_class = reslt1["predic"].to_numpy()

In [42]:
Accuracy = sum(np.array(pred_class)==np.array(true_class))/float(len(true_class))
print(Accuracy)

0.9441772945849884


In [43]:
print(true_class)
print(pred_class)
print(true_class.dtype)
print(pred_class.dtype)
true_clas = true_class.astype(str)
pred_clas = pred_class.astype(str)
print(true_clas.dtype)
print(pred_clas.dtype)

['weather__find' 'weather__find' 'weather__find' ... 'alarm__snooze_alarm'
 'weather__checkSunrise' 'weather__checkSunrise']
['weather__find' 'weather__find' 'weather__find' ... 'alarm__set_alarm'
 'weather__checkSunrise' 'weather__checkSunrise']
object
object
<U25
<U25


In [44]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cf_matrix = confusion_matrix(true_clas, pred_clas)
print(cf_matrix)

[[ 232    1   10    0    3    0    0    3    1    0    0    0    0]
 [   3   74   28    1    2    0    0    0   10    0    0    0    9]
 [   6    4  717    3    4    1    0    0   11    0    0    0    0]
 [   2    0    5  105    0    3    0    0    0    3    0    0    0]
 [   9    0   33    1   58    0    0    0    0    0    0    0    0]
 [   0    0    0    5    0   53    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    1]
 [   8    0    0    0    0    0    0  143   13    0    0    0    0]
 [   0    0   31    0    1    0    0    5  791    2    0    0    1]
 [   0    0    0    5    0    2    0    2   14   96    0    0    1]
 [   0    0    0    0    0    0    0    0    0    0   55    5    0]
 [   0    0    0    0    0    0    0    0    0    0    2   56    5]
 [   0    0    1    2    0    0    0    0    1    1    1    2 2136]]


In [45]:
from sklearn import metrics
print(metrics.classification_report(true_clas, pred_clas))

                           precision    recall  f1-score   support

      alarm__cancel_alarm       0.89      0.93      0.91       250
      alarm__modify_alarm       0.94      0.58      0.72       127
         alarm__set_alarm       0.87      0.96      0.91       746
       alarm__show_alarms       0.86      0.89      0.88       118
      alarm__snooze_alarm       0.85      0.57      0.69       101
alarm__time_left_on_alarm       0.90      0.91      0.91        58
                      nan       0.00      0.00      0.00         1
reminder__cancel_reminder       0.93      0.87      0.90       164
   reminder__set_reminder       0.94      0.95      0.95       831
 reminder__show_reminders       0.94      0.80      0.86       120
    weather__checkSunrise       0.95      0.92      0.93        60
     weather__checkSunset       0.89      0.89      0.89        63
            weather__find       0.99      1.00      0.99      2144

                 accuracy                           0.94    

C:\Users\senug\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\senug\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\senug\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
